In [ ]:
import numpy as np
import cv2
from rembg import remove

[For Display] Raw + Reference image.

In [4]:
img_raw = cv2.imread('./test/Castoria_002.jpg')
img_ref = cv2.imread('./test/reference_raw.png')
h1, w1 = img_raw.shape[:2]
h2, w2 = img_ref.shape[:2]

vis = np.zeros((max(h1, h2), w1+w2,3), np.uint8)

vis[:h1, :w1,:3] = img_raw
vis[:h2, w1:w1+w2,:3] = img_ref
cv2.imwrite('./test/ref_raw.png', vis)

cv2.imshow('output_1', vis)

cv2.waitKey(0)
cv2.destroyAllWindows()

Denoise step.

In [3]:
pth_base = '/Users/wz/Library/CloudStorage/OneDrive-KyotoUniversity/Project/Github/projLite-buffer/hist_matching'
pth_raw = pth_base + '/Castoria_045.jpg'
output = cv2.imread(pth_raw)
sav_pth = pth_base + '/deno.png'
output = output[1732-1142:-1, :] #* Cropping step.
h = 10
#* Size in pixels of the template patch that is used to compute weights. Should be odd. Recommended value 7 pixels
templateWindowSize = 7
#* Size in pixels of the window that is used to compute weighted average for given pixel. Should be odd. Affect performance linearly: greater searchWindowsSize - greater denoising time. Recommended value 21 pixels
searchWindowSize = 21
output_ = cv2.fastNlMeansDenoisingColored(output, None, h, h, templateWindowSize, searchWindowSize)
cv2.imwrite(sav_pth, output_)

cv2.imshow('output_1', output_)

cv2.waitKey(0)
cv2.destroyAllWindows()

Cutout step.

In [4]:
input_path = sav_pth
deno_cut_pth = pth_base + '/deno_cutout.png'

with open(input_path, 'rb') as i:
    with open(deno_cut_pth, 'wb') as o:
        input = i.read()
        output = remove(input)
        o.write(output)

Histogram equilization + matching step.

In [5]:
def hist_match(img_raw_, img_ref_):

    for i in range(img_raw_.shape[2]):
        img_raw, img_ref = img_raw_[:,:,i], img_ref_[:,:,i]
        img_raw_shape = img_raw.shape
        
        img_raw, img_ref = img_raw.flatten(), img_ref.flatten()

        raw_values, raw_idx, raw_num = np.unique(img_raw, return_inverse=True,return_counts=True)
        ref_values, ref_num = np.unique(img_ref, return_counts=True)

        raw_cdf = np.cumsum(raw_num).astype(np.float64) #* Cumulative sum
        raw_cdf /= raw_cdf[-1] #* Normalization
        ref_cdf = np.cumsum(ref_num).astype(np.float64)
        ref_cdf /= ref_cdf[-1]

        #* T(ref) >= sum(p(s))
        interp_t_val = np.interp(raw_cdf, ref_cdf, ref_values)
        img_raw_[:,:,i] = interp_t_val[raw_idx].reshape(img_raw_shape)
        
    return img_raw_

pth_raw = deno_cut_pth
pth_ref = '/Users/wz/Library/CloudStorage/OneDrive-KyotoUniversity/Project/Github/projLite-buffer/hist_matching/ref.png'
pth_hist = pth_base + '/deno_cut_histm.png'
img_raw = cv2.imread(pth_raw)
img_ref = cv2.imread(pth_ref)
img_raw = hist_match(img_raw, img_ref)

cv2.imwrite(pth_hist, img_raw)

True

Dual-denoise.

In [15]:
output = cv2.imread(pth_hist)
hist_deno = pth_base + '/histm_deno.png'
h = 3
#* Size in pixels of the template patch that is used to compute weights. Should be odd. Recommended value 7 pixels
templateWindowSize = 7
#* Size in pixels of the window that is used to compute weighted average for given pixel. Should be odd. Affect performance linearly: greater searchWindowsSize - greater denoising time. Recommended value 21 pixels
searchWindowSize = 21
output_ = cv2.fastNlMeansDenoisingColored(output, None, h, h, templateWindowSize, searchWindowSize)
cv2.imwrite(hist_deno, output_)

cv2.imshow('output_1', output_)

cv2.waitKey(0)
cv2.destroyAllWindows()

Adjust alpha.

In [20]:
mask = cv2.imread(deno_cut_pth) #* deno_cutout
mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
# Apply a threshold to the grayscale image
_, threshold = cv2.threshold(mask, 0, 255, cv2.THRESH_BINARY)
t_x, t_y = np.where(threshold<=1)
t_ind = np.concatenate((t_x.reshape(-1,1), t_y.reshape(-1,1)), axis=1) #* Return transparent coordinates.

new = cv2.imread(hist_deno)
new = cv2.cvtColor(new, cv2.COLOR_BGR2RGBA)
sav_new = pth_base + '/alpha.png'
alpha = new[:, :, 3]
alpha[t_x, t_y] = 0
new[:, :, 3] = alpha
new = cv2.cvtColor(new, cv2.COLOR_RGBA2BGRA)

cv2.imwrite(sav_new, new)

cv2.imshow('inpaint', new)

cv2.waitKey(0)
cv2.destroyAllWindows()

To remove some noise.

In [22]:
input_path = sav_new
deno_cut_pth = pth_base + '/alpha_cutout.png'

with open(input_path, 'rb') as i:
    with open(deno_cut_pth, 'wb') as o:
        input = i.read()
        output = remove(input)
        o.write(output)

[For display] Compare

In [8]:
img_raw = cv2.imread('./test/hist_match_deno_cutout.png')
img_ref = cv2.imread('./test/reference_raw.png')
h1, w1 = img_raw.shape[:2]
h2, w2 = img_ref.shape[:2]

vis = np.zeros((max(h1, h2), w1+w2,3), np.uint8)

vis[:h1, :w1,:3] = img_raw
vis[:h2, w1:w1+w2,:3] = img_ref
cv2.imwrite('./test/match_raw.png', vis)

cv2.imshow('output_1', vis)

cv2.waitKey(0)
cv2.destroyAllWindows()